# Code to do estimations of resources for AWS
- Corresponding to experiment 1: PoC: Groundstate of LiH with ActiveSpaceOptimization on sv

In [1]:
from braket.tracking import Tracker
t = Tracker().start()

In [2]:
from pprint import pprint

import matplotlib.pyplot as plt
from braket.aws import AwsDevice
from braket.devices import Devices
from braket.jobs import hybrid_job, save_job_result
from qiskit.circuit.library import TwoLocal
from qiskit.primitives import BackendEstimator
from qiskit_algorithms.optimizers import SLSQP

from qiskit_algorithms.minimum_eigensolvers import NumPyMinimumEigensolver, VQE
from qiskit_nature.second_q.transformers import FreezeCoreTransformer
from qiskit_nature.second_q.formats.molecule_info import MoleculeInfo
from qiskit_nature.second_q.mappers import ParityMapper, JordanWignerMapper
from qiskit_nature.second_q.circuit.library import UCCSD, HartreeFock

from qiskit_nature.second_q.drivers import PySCFDriver

from qiskit_aer.primitives import Estimator as AerEstimator

from qiskit_braket_provider import BraketProvider, BraketLocalBackend, to_braket
import time
from datetime import datetime

import numpy as np

In [3]:
NQUBITS = 2
MAXITER = 1000
SHOTS = 1000
DIST = 1.6

## Normal with Jordan-Wigner and TwoQubitReduction + FreezeCore + ActiveSpace

In [4]:
def get_qubit_op_as(dist):
    # Define Molecule
    molecule = MoleculeInfo(
        # Coordinates in Angstrom
        symbols=["Li", "H"],
        coords=([0.0, 0.0, 0.0], [dist, 0.0, 0.0]),
        multiplicity=1,  # = 2*spin + 1
        charge=0,
    )

    driver = PySCFDriver.from_molecule(molecule)
    problem = driver.run()

    # Now you can get the reduced electronic structure problem
    from qiskit_nature.second_q.transformers import FreezeCoreTransformer

    fc_transformer = FreezeCoreTransformer()

    fc_problem = fc_transformer.transform(problem)

    # active space transformer
    from qiskit_nature.second_q.transformers import ActiveSpaceTransformer

    as_transformer = ActiveSpaceTransformer(2, 2)

    as_problem = as_transformer.transform(fc_problem)
    
    num_particles = as_problem.num_particles
    num_spatial_orbitals = as_problem.num_spatial_orbitals

    mapper = ParityMapper(num_particles=num_particles)
    qubit_op = mapper.map(as_problem.second_q_ops()[0])
    print (f"Problem spatial orbitals {problem.num_spatial_orbitals}")
    print (f"Problem particles {problem.num_particles}")
    print (f"FC-Problem spatial orbitals {fc_problem.num_spatial_orbitals}")
    print (f"FC-Problem particles {fc_problem.num_particles}")
    print (f"AS-Problem spatial orbitals {as_problem.num_spatial_orbitals}")
    print (f"AS-Problem particles {as_problem.num_particles}")
    return qubit_op, as_problem.num_particles, as_problem.num_spatial_orbitals, as_problem, mapper

In [5]:
def exact_solver(qubit_op, problem):
    sol = NumPyMinimumEigensolver().compute_minimum_eigenvalue(qubit_op)
    result = problem.interpret(sol)
    return result

## Pure Qiskit and Local

In [6]:
distances = [DIST] #np.arange(1.5, 3.0, 0.1)
exact_energies = []
vqe_energies = []
optimizer = SLSQP(maxiter=MAXITER)
noiseless_estimator = AerEstimator(approximation=True)

# pylint: disable=undefined-loop-variable,line-too-long
for dist in distances:
    (qubit_op, num_particles, num_spatial_orbitals, problem, mapper) = get_qubit_op_as(
        dist
    )

    result = exact_solver(qubit_op, problem)
    exact_energies.append(result.total_energies[0].real)
    init_state = HartreeFock(num_spatial_orbitals, num_particles, mapper)
    var_form = UCCSD(
        num_spatial_orbitals, num_particles, mapper, initial_state=init_state
    )
    print(f"Qubits needed in ansatz: {var_form.num_qubits}")
    vqe = VQE(
        noiseless_estimator,
        var_form,
        optimizer,
        initial_point=[0] * var_form.num_parameters,
    )
    vqe_calc = vqe.compute_minimum_eigenvalue(qubit_op)
    vqe_result = problem.interpret(vqe_calc).total_energies[0].real
    vqe_energies.append(vqe_result)
    print(
        f"Interatomic Distance: {np.round(dist, 2)}",
        f"VQE Result: {vqe_result:.5f}",
        f"Exact Energy: {exact_energies[-1]:.5f}",
        f"Cost function evals: {vqe_calc.cost_function_evals}",
        f"Optimizer evals: {vqe_calc.optimizer_evals}",
    )

print("All energies have been calculated")

Problem spatial orbitals 6
Problem particles (2, 2)
FC-Problem spatial orbitals 5
FC-Problem particles (1, 1)
AS-Problem spatial orbitals 2
AS-Problem particles (1, 1)
Qubits needed in ansatz: 2
Interatomic Distance: 1.6 VQE Result: -7.86213 Exact Energy: -7.86213 Cost function evals: 20 Optimizer evals: None
All energies have been calculated


## Braket DM1

In [7]:
backend = BraketProvider().get_backend("dm1")

/tmp/ipykernel_15078/701233080.py:1: DeprecationWarning: The class ``qiskit.providers.provider.Provider`` is deprecated as of qiskit 1.1. It will be removed no earlier than 3 months after the release date. The abstract Provider and ProviderV1 classes are deprecated and will be removed in 2.0. You can just remove it as the parent class and a `get_backend` method that returns the backends from `self.backend`.
  backend = BraketProvider().get_backend("dm1")
/tmp/ipykernel_15078/701233080.py:1: DeprecationWarning: The method ``qiskit.providers.provider.ProviderV1.get_backend()`` is deprecated as of qiskit 1.1. It will be removed no earlier than 3 months after the release date. The abstract Provider and ProviderV1 classes are deprecated and will  be removed in 2.0. You can just remove it as the parent class and a `get_backend` method that returns the backends from `self.backend`.
  backend = BraketProvider().get_backend("dm1")


In [8]:
#backend = BraketLocalBackend()

In [9]:
estimator = BackendEstimator(backend=backend, options={"shots": SHOTS})

In [10]:
slsqp = SLSQP(maxiter=MAXITER)

In [11]:
distances = [DIST] #np.arange(1.5, 3.0, 0.1)
vqe_energies = []
optimizer = SLSQP(maxiter=MAXITER)

start = time.time()
# pylint: disable=undefined-loop-variable,line-too-long
for dist in distances:
    (qubit_op, num_particles, num_spatial_orbitals, problem, mapper) = get_qubit_op_as(
        dist
    )
    init_state = HartreeFock(num_spatial_orbitals, num_particles, mapper)
    var_form = UCCSD(
        num_spatial_orbitals, num_particles, mapper, initial_state=init_state
    )
    print(f"Qubits needed in ansatz: {var_form.num_qubits}")
    vqe = VQE(
        estimator,
        var_form,
        slsqp,
        initial_point=[0] * var_form.num_parameters,
    )
    vqe_calc = vqe.compute_minimum_eigenvalue(qubit_op)
    vqe_result = problem.interpret(vqe_calc).total_energies[0].real
    vqe_energies.append(vqe_result)
    print(
        f"Interatomic Distance: {np.round(dist, 2)}",
        f"VQE Result: {vqe_result:.5f}",
        f"Cost function evals: {vqe_calc.cost_function_evals}",
        f"Optimizer evals: {vqe_calc.optimizer_evals}",
    )
    
end = time.time()
# print execution time
print('Code execution time [sec]:', end - start)

Problem spatial orbitals 6
Problem particles (2, 2)
FC-Problem spatial orbitals 5
FC-Problem particles (1, 1)
AS-Problem spatial orbitals 2
AS-Problem particles (1, 1)
Qubits needed in ansatz: 2


/home/marc/.local/lib/python3.10/site-packages/qiskit_braket_provider/providers/adapter.py:489: UserWarning: Device does not support global phase; global phase of 5.497787143782138 will not be included in Braket circuit
  warnings.warn(


Interatomic Distance: 1.6 VQE Result: -7.73095 Cost function evals: 183 Optimizer evals: None
Code execution time [sec]: 646.5522718429565


In [12]:
print("Quantum Task Summary")
print(t.quantum_tasks_statistics())
print(f"Estimated cost to run this example: {t.qpu_tasks_cost() + t.simulator_tasks_cost()} USD")

Quantum Task Summary
{'arn:aws:braket:::device/quantum-simulator/amazon/sv1': {'shots': 728000, 'tasks': {'COMPLETED': 728}, 'execution_duration': datetime.timedelta(seconds=7, microseconds=120000), 'billed_execution_duration': datetime.timedelta(seconds=2184)}}
Estimated cost to run this example: 2.7300000000 USD
